<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/moduler_optimalisering_evaulering/moduler_optimalisering_evaluering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importering av nødvendige bibliotek og pakker

In [ ]:
import pandas as pd
import numpy as np
import copy
import math
import time
from statistics import mean, stdev

from sklearn.model_selection import train_test_split

Evalueringsmetrikker

In [ ]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             r2_score)

# Lager en funksjon for RMSE
def rmse(y_sann, y_pred):

  return np.sqrt(mean_squared_error(y_sann, y_pred))

Tilfeldighetsfrø

In [ ]:
#Tilfeldighetsfrø
random_seed = 2024

# Oppdeling av trening og testsett

## Alternativ 1: Fordelt på Enzymtyper

In [ ]:
def trening_testsett_oppdeling_enzym(rå_data: pd.DataFrame,
                                     filtert_df: pd.DataFrame,
                                     test_andel: float = 0.2,
                                     random_seed: int = 2024):

  """
  Deler opp datasett i trenings- og testsett, med hensyn til lik fordeling i
  ensymtyper.

  Args:
      - rå_data: Originalt datasett med alle observasjoner og egeskaper
      (type: pd.DataFrame)
      - filtrert_df: Et subsett av data med kun responsvariabel (type: pd.DataFrame)
      - test_andel: Andel av data som skal benyttes til testsett (type: float).
      Standardverdi: 0.2
      - random_seed: Seed for tilfeldig oppdeling (type: int).
      Standardverdi: 123

  Returns:
        - trening_markert: Treningssett med markeringer (type: pd.DataFrame).
        - enzymtyper_trening: Enzymkoder for treningssettet (type: pd.Series).
        - test_markert: Testsett med markeringer (type: pd.DataFrame).
        - enzymtyper_test: Enzymkoder for testsettet (type: pd.Series).
        - trening_umarkert: Treningssett uten markeringer  (type: pd.DataFrame).
        - enzymtyper_umarkert: Enzymkoder for treningssett uten markeringer (type: pd.Series).
        - treningsdata: Fullstendig treningssett (markert og umarkerte data) (type: pd.DataFrame).
  """


  # Responsvariabel
  responsvariabel = filtert_df.columns[-1]

  # Deler opp markert og umarkert del av datasettet
  data_markert = filtert_df.dropna(subset=[responsvariabel])
  data_umarkert = filtert_df[filtert_df[responsvariabel].isna()]

  # Enzymtyper markert data
  enzym_data = rå_data.merge(data_markert,
                                 left_index=True,
                                 right_index=True,
                                 how='inner',
                                 suffixes=('','_drop'))['EnzymeCode']

  # Splitter trening og testdata etter enzymtype
  trening_markert, test_markert,\
  enzymtyper_trening, enzymtyper_test = train_test_split(data_markert,
                                                         enzym_data,
                                                         test_size=test_andel,
                                                         stratify=enzym_data,
                                                         random_state=random_seed)

  # Definerer trening og testsett for data

  felles_indeks = filtert_df.index.intersection(test_markert.index)
  treningsdata = filtert_df.drop(felles_indeks)
  trening_umarkert = treningsdata[treningsdata[responsvariabel].isna()]

  # Enzymtype for umarkert data
  enzymtyper_umarkert = rå_data.merge(trening_umarkert,
                                          left_index=True,
                                          right_index=True,
                                          how='inner',
                                          suffixes=('','_drop'))['EnzymeCode']



  return trening_markert, enzymtyper_trening, test_markert, enzymtyper_test, trening_umarkert, enzymtyper_umarkert, treningsdata

## Alternativ 2: Manuell fordeling

In [ ]:
def trening_testsett_oppdeling_dag(rå_data: pd.DataFrame,
                                   filtert_df: pd.DataFrame,
                                   test_andel: float = 0.2,
                                   test_ekskluder_kalkun_kylling: bool = True):

  """
  Deler opp datasettet i trenings- og testsett, med hensyn til daglige og
  ukentlige variasjoner. Utelukker råmaterialeblanding "Turkey" og "Chicken"
  fra testsettet.

  Args:
      - rå_data: Originalt datasett med alle observasjoner og egeskaper
      (type: pd.DataFrame)
      - filtrert_df: Et subsett av data med kun responsvariabel (type: pd.DataFrame)
      - test_andel: Andel av data som skal benyttes til testsett, (type: float)
      - test_ekskluder_kalkun_kylling: Bestemmer om kravet om ekskludering av
      råmaterialeblandingene "Turkey" og "Chicken" skal overholdes (type: bool)

  Returns:
      - trening_markert: Treningssett med markeringer (type: pd.DataFrame)
      - test_markert: Testsett med markeringer (type: pd.DataFrame)
      - trening_umarkert: Treningssett uten markeringer (type: pd.DataFrame)
      - treningsdata: Fullstendig treningssett (type: pd.DataFrame)
  """

  # Responsvariabel
  responsvariabel = filtert_df.columns[-1]

  # Lager nytt trening og testsett
  data_markert = filtert_df.dropna(subset=[responsvariabel]).copy()

  trening_markert = copy.deepcopy(data_markert)
  test_markert = pd.DataFrame(columns=list(data_markert.columns))

  # Råmateriale type
  råmateriale_data = rå_data.merge(data_markert,
                                  left_index=True,
                                  right_index=True,
                                  how='inner',
                                  suffixes=('','_drop'))['RawMaterialMix']

  data_markert.loc[:, 'RawMaterialMix'] = råmateriale_data.copy()


  # Går gjennom hver dag, hver uke og samler første andel av rader til testsett
  for uke in data_markert.index.isocalendar().week.unique():
      for dag in data_markert.index.isocalendar().day.unique():
          data = data_markert[(data_markert.index.isocalendar().week == uke) &
                    (data_markert.index.isocalendar().day == dag)]
          test_rader = round((data.shape[0] * test_andel))
          test_data_dag = data.head(test_rader)

          if test_ekskluder_kalkun_kylling:

            # Sjekker første instans av "Turkey" eller "Chicken" i foreløpig testsett
            krav = (test_data_dag["RawMaterialMix"] == "Turkey") | (test_data_dag["RawMaterialMix"] == "Chicken")

            # Hvis testsettet inneholder "Turkey" eller "Chicken", legges det
            # til en rad, da første rad med gjelende utelukkes
            if krav.any():
              test_rader += 1

              # Finner mengde testsett uten "Turkey" eller "Chicken"
              while krav.any():
                test_data_dag = data.loc[krav.idxmax():].head(test_rader).copy()
                test_data_dag = test_data_dag.drop(index = krav.idxmax())

                test_rader -=1


                if (test_rader == 0)| (test_data_dag.shape[0] == 0):
                  pass

                # Finner nest beste mengde testsett uten "Turkey" eller "Chicken"
                elif (test_data_dag.shape[0] < test_rader) & (test_data_dag.index[-1] == data.index[-1]):
                  test_rader -= 1
                  test_data_dag = data.head(test_rader).copy()

                krav = (test_data_dag["RawMaterialMix"] == "Turkey") | (test_data_dag["RawMaterialMix"] == "Chicken")

                test_rader +=1

          # Dropper "RawMaterialMix" som kolonne
          test_data_dag = test_data_dag.drop(columns = ['RawMaterialMix'])

          # Legger til i fullstendig testsett og fjerner fra treningssett
          test_markert = pd.concat([test_markert, test_data_dag])
          trening_markert = trening_markert.drop(test_data_dag.index.copy())

  felles_indeks = filtert_df.index.intersection(test_markert.index)
  treningsdata = filtert_df.drop(felles_indeks.copy())
  trening_umarkert = treningsdata[treningsdata[responsvariabel].isna()]

  return trening_markert, test_markert, trening_umarkert, treningsdata

## Alternativ 2: Manuell trening og valideringssett

In [ ]:
def trening_validering_oppdeling_dag(rå_data: pd.DataFrame,
                                     filtert_df: pd.DataFrame,
                                     n_splitt: int,
                                     test_ekskluder_kalkun_kylling: bool = True):
  """
  Deler opp gitt data med hensyn til ukentklig og daglig variasjon,
  i trening- og valideringssett for kryssvalidering.
  Utelukker råmaterialeblanding "Turkey" og "Chicken" fra valideringssettet.

  Args:
      - rå_data: Originalt datasett med alle observasjoner og egeskaper
      (type: pd.DataFrame)
      - filtrert_df: Et subsett av data med kun responsvariabel
      (type: pd.DataFrame)
      - test_andel: Andel av data som skal benyttes til testsett (type: float)
      - test_ekskluder_kalkun_kylling: Bestemmer om kravet om ekskludering av
      råmaterialeblandingene "Turkey" og "Chicken" skal overholdes (type: bool)

  Returns:
      - treningssett_index: Indekser til treningssettene for hver fold i
      kryssvalidering (type: Dict[int, pd.Index])
      - valideringssett_index: Indekser til valideringssettene for hver fold i
      kryssvalidering (type: Dict[int, pd.Index])
  """

  # Responsvariabel
  responsvariabel = filtert_df.columns[-1]

  # Lager nytt trening og testsett
  data_markert = filtert_df.dropna(subset=[responsvariabel]).copy()



  # Råmateriale type
  råmateriale_data = rå_data.merge(data_markert,
                                        left_index=True,
                                        right_index=True,
                                        how='inner',
                                        suffixes=('','_drop'))['RawMaterialMix']

  data_markert.loc[:, 'RawMaterialMix'] = råmateriale_data.copy()



  # For å lagre indeks til hvert "fold"
  treningssett_index = dict()
  valideringssett_index = dict()

  fold_rest = n_splitt
  for i in range(n_splitt):

    # Ønsket størrelse på hvert validerings "fold"
    test_andel = 1/fold_rest

    trening_markert = copy.deepcopy(data_markert)
    val_markert = pd.DataFrame(columns=list(data_markert.columns))

    data_markert_filtrert = data_markert

    # Neste "fold" skal ikke inneholde data fra tidligere "folds"
    if i > 0:
      for k in range(i):
        data_markert_filtrert = data_markert_filtrert.drop(valideringssett_index[k])

    # Går gjennom hver dag, hver uke og samler første andel av rader til testsett
    for uke in data_markert_filtrert.index.isocalendar().week.unique():
        for dag in data_markert_filtrert.index.isocalendar().day.unique():
          data = data_markert_filtrert[(data_markert_filtrert.index.isocalendar().week == uke) &
                      (data_markert_filtrert.index.isocalendar().day == dag)]

          test_rader = math.floor((data.shape[0] * test_andel))

          test_data_dag = data.head(test_rader)

          if test_ekskluder_kalkun_kylling:
            # Sjekker første instans av "Turkey" eller "Chicken" i foreløpig testsett
            krav = (test_data_dag["RawMaterialMix"] == "Turkey") | (test_data_dag["RawMaterialMix"] == "Chicken")

            # Hvis testsettet inneholder "Turkey" eller "Chicken", legges det
            # til en rad, da første rad med gjelende utelukkes
            if krav.any():

              # Legger til en rad, da øverste rad vil være "Turkey"/"Chicken",
              # som vil fjernes
              test_rader += 1

              # Finner mengde testsett uten "Turkey" eller "Chicken"
              while krav.any():
                test_data_dag = data.loc[krav.idxmax():].head(test_rader).copy()
                test_data_dag = test_data_dag.drop(index = krav.idxmax())

                # Reduserer antall testrader til utgangspunktet
                test_rader -=1

                if (test_rader == 0)| (test_data_dag.shape[0] == 0) :
                  pass

                # Finner nest beste mengde testsett uten "Turkey" eller "Chicken",
                # med -1 testrader enn utgangspunktet
                elif (test_data_dag.shape[0] < test_rader) & (test_data_dag.index[-1] == data.index[-1]):
                  test_rader -= 1
                  test_data_dag = data.head(test_rader).copy()

                krav = (test_data_dag["RawMaterialMix"] == "Turkey") | (test_data_dag["RawMaterialMix"] == "Chicken")

                test_rader +=1

          # Legger til i fullstendig testsett og fjerner fra treningssett
          val_markert = pd.concat([val_markert, test_data_dag])
          trening_markert = trening_markert.drop(test_data_dag.index.copy())

    # Legger til indeks for trening og validering for hver "fold"
    treningssett_index[i] = trening_markert.index
    valideringssett_index[i] = val_markert.index

    fold_rest -= 1

  return treningssett_index, valideringssett_index

# Diverse moduler

## Alternativ 2: Umarkert andel

In [ ]:
def umarkert_andel_dag(treningssett_umarkert: pd.DataFrame,
                       umarkert_andel: float =0.25):

  """
  Deler opp datasett med umerket data i benyttet og ubenyttet del

  Args:
      - treningssett_umarkert: Datasett med umarkert data (type: pd.DataFrame)
      - umarkert_andel: Andel av umarkert data som skal benyttes

 Returns:
      - trening_umarkert_filtrert: Umarkert data som benyttes
      (type: pd. DataFrame)
      - trening_umarkert_rest : Resterende umarkert data som ikke benyttes
      (type: pd.DataFrame)

  """

  # Deler opp datasettet i benyttet og ubenyttet del
  trening_umarkert_rest = copy.deepcopy(treningssett_umarkert)
  trening_umarkert_filtrert = pd.DataFrame(columns=list(treningssett_umarkert.columns))


  # Går gjennom hver dag, hver uke og samler første andel av rader til benyttet sett
  for uke in treningssett_umarkert.index.isocalendar().week.unique():
      for dag in treningssett_umarkert.index.isocalendar().day.unique():

          data = treningssett_umarkert[(treningssett_umarkert.index.isocalendar().week == uke) &
                    (treningssett_umarkert.index.isocalendar().day == dag)]

          # Finner antall rader for den benyttede delen
          umarkert_andel_rader = round((data.shape[0] * umarkert_andel))

          # Velger rader for den benyttede delen
          umarkert_andel_data_dag = data.head(umarkert_andel_rader)

          # Legger til i benyttet del og fjerner fra resterende
          trening_umarkert_filtrert = pd.concat([trening_umarkert_filtrert, umarkert_andel_data_dag])
          trening_umarkert_rest = trening_umarkert_rest.drop(umarkert_andel_data_dag.index.copy())

  return trening_umarkert_filtrert, trening_umarkert_rest

## Enzymtype informasjon

In [ ]:
def enzymfordeling(rådata: pd.DataFrame,
                   data: pd.DataFrame):

  """
  Viser fordelingen av enzymtyper i gjeldende data med frekvens og andel

  Args:
      - rådata: Originalt datasett med alle observasjoner og egeskaper
      (type: pd.DataFrame)
      - data: Subsett av data med en responsvariabel (type: pd.DataFrame)

  Returns:
      - enzymtyper_data: Frekvens og andel for enzymkoder (type: pd.DataFrame)

  """

  # Enzymtyper markert data
  enzym_data = rådata.merge(data,
                            left_index=True,
                            right_index=True,
                            how='inner',
                            suffixes=('','_drop'))['EnzymeCode']

  # Beregner frekvens og andel
  frekvens = enzym_data.value_counts()
  andel = round(enzym_data.value_counts(normalize=True) * 100, 2)

  # Samler frekvens og andel i en DataFrame
  enzymtyper_data = pd.DataFrame({
      'Frekvens': frekvens,
      'Andel': andel
      })

  return enzymtyper_data

# Splitt etter min enzymtype antall

In [ ]:
def n_splitt_enzym(rå_data,
                   data_markert):

  enzym_markert = enzymfordeling(rå_data,
                                 data_markert)

  enzymtype_min = enzym_markert['Frekvens'].min()

  n_splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  return n_splitt

# Andre Moduler

In [ ]:
def evaluering_beregning(y_sann, y_pred):

  rmse_verdi = rmse(y_sann, y_pred)
  r2_verdi = r2_score(y_sann, y_pred)
  mae_verdi = mean_absolute_error(y_sann, y_pred)
  mape_verdi = mean_absolute_percentage_error(y_sann, y_pred)

  return rmse_verdi, r2_verdi, mae_verdi, mape_verdi

In [ ]:
class Resultat_evaluering:

  """
  Klasse for lagring og håndtering av diverse evaularingsresultater for modell

  Attributter:
      - rmse_trening_resultat (list:[float]): Liste med RMSE-verdier for
      treningssettresultater.
      - r2_trening_resultat (list:[float]): Liste med R2-verdier for
      treningssettresultater.
      - mae_trening_resultat (list:[float]): Liste med MAE-verdier for treningssettresultater.

      - mape_trening_resultat: Liste med MAPE-verdier for treningssettresultater.
      (type: list:[float])
      - rmse_test_resultat: Liste med RMSE-verdier for testsettresultater.

      (type: list:[float])
      - r2_test_resultat: Liste med R2-verdier for testsettresultater.
      (type: list:[float])
      - mae_test_resultat: Liste med MAE-verdier for testsettresultater.
      (type: list:[float])
      - mape_test_resultat: Liste med MAPE-verdier for testsettresultater.
      (type: list:[float])


  """

  def __init__(self):

    self.rmse_trening_res = []
    self.r2_trening_res = []
    self.mae_trening_res = []
    self.mape_trening_res = []

    self.rmse_val_res = []
    self.r2_val_res = []
    self.mae_val_res = []
    self.mape_val_res = []

    self.rmse_test_res = []
    self.r2_test_res = []
    self.mae_test_res = []
    self.mape_test_res = []


  def evaluering(self,
                 y_trening,
                 y_trening_pred,
                 y_val,
                 y_val_pred,
                 y_test,
                 y_test_pred):

    # Trening
    rmse_trening, r2_trening, mae_trening, mape_trening\
    = evaluering_beregning(y_trening,
                           y_trening_pred)

    # Validering
    rmse_val, r2_val, mae_val, mape_val\
    = evaluering_beregning(y_val,
                           y_val_pred)

    # Test
    rmse_test, r2_test, mae_test, mape_test\
    = evaluering_beregning(y_test,
                           y_test_pred)

    # Fra R2 dokumentasjon i scikit-learn: Nan er erstattet med 1.0.
    # Skal gjøres som standard, men oppstod problemer underveis.
    if np.isnan(r2_test):
      r2_test = 1.0
    if np.isnan(r2_val):
      r2_val = 1.0

    self.rmse_trening_res.append(rmse_trening)
    self.r2_trening_res.append(r2_trening)
    self.mae_trening_res.append(mae_trening)
    self.mape_trening_res.append(mape_trening)

    self.rmse_val_res.append(rmse_trening)
    self.r2_val_res.append(r2_trening)
    self.mae_val_res.append(mae_trening)
    self.mape_val_res.append(mape_trening)

    self.rmse_test_res.append(rmse_test)
    self.r2_test_res.append(r2_test)
    self.mae_test_res.append(mae_test)
    self.mape_test_res.append(mape_test)

In [ ]:
# Funksjon for lagring av resultater av test
def res_eval_opt(dict_res_opt,
                 modell,
                 test_andel,
                 trening_andel,
                 parametere,
                 rmse_score_opt,
                 resultater,
                 umarkert_antall=None,
                 umarkert_benyttet=None):

  """

  Funksjon som lagrer resultater fra testsett i en dictionary

  Args:
      - dict_res_opt (dict): Dictionary der resultater lagres.
      - modell (str): Navn på gjeldende modell.
      - test_andel (float): Andel av data satt av til test.
      - trening_andel (float): Andel av treningsdata benyttet til trening.

      - rmse_score_opt ()
      - parametere: Dictionary med parametere fra hyperoptimalisering av
      modellen. (type: dict)
      - resultater: Liste med resultater fra optimalisering av modellen på
      treningsdata og evaluering av modellen på testdata
      (RMSE, R2, MAE, MAPE, n_runder, n_splitt, tid_trening). (type: list)
      - umarkert_antall: Totalt antall umarkerte datapunkter. (type: int)
      - umarkert_benyttet: Antall umarkerte datapunkter benyttet av modellen.
      (type: int)

  Funksjonalitet:
    - Legger til navn på modell og verdier for test_andel, trening_andel,
    trening_markert_antall, umarkert_antall, umarkert_benyttet,
    tid_trening, parametere, RMSE_test, R2_test, MAE_test, MAPE_test,
    n_runder, og n_splitt til dict_res.

  """

  # Hvis semi-veiledet modell
  # Hvis semi-veiledet modell
  if umarkert_antall:

    dict_res_opt['umarkert_antall'].append(umarkert_antall)

    if umarkert_benyttet:
      dict_res_opt['umarkert_benyttet'].append(umarkert_benyttet)

    else:
      dict_res_opt['umarkert_benyttet'].append(0)

  dict_res_opt['modell'].append(modell)
  dict_res_opt['test_andel'].append(test_andel)
  dict_res_opt['trening_andel'].append(trening_andel)
  dict_res_opt['trening_markert_antall'].append(resultater[8])
  dict_res_opt['tid_trening'].append(resultater[9])
  dict_res_opt['parametere'].append(parametere)
  dict_res_opt['RMSE_opt'].append(rmse_score_opt)
  dict_res_opt['RMSE_test'].append(resultater[0])
  dict_res_opt['R2_test'].append(resultater[1])
  dict_res_opt['MAE_test'].append(resultater[2])
  dict_res_opt['MAPE_test'].append(resultater[3])
  dict_res_opt['n_runder'].append(resultater[10])
  dict_res_opt['n_splitt'].append(resultater[11])
  dict_res_opt['tid_optimalisering'].append(resultater[12])




# Funksjon for lagring av resultater av trening
def res_eval(dict_res,
             modell,
             test_andel,
             trening_andel,
             trening_markert_antall,
             parametere,
             rmse_score_opt,
             n_splitt,
             res_eval,
             umarkert_antall=None,
             umarkert_benyttet=None):

  """

  Funksjon som lagrer resultater fra kryssvalidering med trening- og
  valideringssett, i en dictionary

  Args:
      - dict_res: Dictionary der resultatene lagres. (type: dict)
      - modell: Navn på modell. (type: str)
      - test_andel: Andel av dataen satt av til test. (type: float)
      - trening_andel: Andel av treningsdata som benyttes til trening.
      (type: float)
      - umarkert_antall: Totalt antall umarkerte datapunkter. (type: int)
      - umarkert_benyttet: Antall umarkerte datapunkter brukt i modellen.
      (type: int)
      - parametere: Dictionary med parametere for modell. (type: dict)
      - n_splitt: Antall oppdelinger av treningsdata i kryssvalidering.
      (type: int)
      - res_eval: En instans av klasse "ResultatEvaluering" som inneholder
      evalueringsresultater. (type: ResultatEvaulering)

  """

  # Hvis semi-veiledet modell
  if umarkert_antall:

    dict_res['umarkert_antall'].append(umarkert_antall)

    if umarkert_benyttet:
      dict_res['umarkert_benyttet'].append(umarkert_benyttet)

    else:
      dict_res['umarkert_benyttet'].append(0)


  dict_res['modell'].append(modell)
  dict_res['test_andel'].append(test_andel)
  dict_res['trening_andel'].append(trening_andel)
  dict_res['trening_markert_antall'].append(trening_markert_antall)
  dict_res['parametere'].append(parametere)
  dict_res['n_splitt'].append(n_splitt)
  dict_res['RMSE_opt'].append(rmse_score_opt)

  # Testresultater
  dict_res['RMSE_test_snitt'].append(round(mean(res_eval.rmse_test_res),3))
  dict_res['RMSE_test_std'].append(round(stdev(res_eval.rmse_test_res),3))
  dict_res['R2_test_snitt'].append(round(mean(res_eval.r2_test_res),3))
  dict_res['R2_test_std'].append(round(stdev(res_eval.r2_test_res),3))
  dict_res['MAE_test_snitt'].append(round(mean(res_eval.mae_test_res),3))
  dict_res['MAE_test_std'].append(round(stdev(res_eval.mae_test_res),3))
  dict_res['MAPE_test_snitt'].append(round(mean(res_eval.mape_test_res),3))
  dict_res['MAPE_test_std'].append(round(stdev(res_eval.mape_test_res),3))

  # Valideringsresulateter
  dict_res['RMSE_val_snitt'].append(round(mean(res_eval.rmse_val_res),3))
  dict_res['RMSE_val_std'].append(round(stdev(res_eval.rmse_val_res),3))
  dict_res['R2_val_snitt'].append(round(mean(res_eval.r2_val_res),3))
  dict_res['R2_val_std'].append(round(stdev(res_eval.r2_val_res),3))
  dict_res['MAE_val_snitt'].append(round(mean(res_eval.mae_val_res),3))
  dict_res['MAE_val_std'].append(round(stdev(res_eval.mae_val_res),3))
  dict_res['MAPE_val_snitt'].append(round(mean(res_eval.mape_val_res),3))
  dict_res['MAPE_val_std'].append(round(stdev(res_eval.mape_val_res),3))

  # Treningssresultater
  dict_res['RMSE_trening_snitt'].append(round(mean(res_eval.rmse_trening_res),3))
  dict_res['RMSE_trening_std'].append(round(stdev(res_eval.rmse_trening_res),3))
  dict_res['R2_trening_snitt'].append(round(mean(res_eval.r2_trening_res),3))
  dict_res['R2_trening_std'].append(round(stdev(res_eval.r2_trening_res),3))
  dict_res['MAE_trening_snitt'].append(round(mean(res_eval.mae_trening_res),3))
  dict_res['MAE_trening_std'].append(round(stdev(res_eval.mae_trening_res),3))
  dict_res['MAPE_trening_snitt'].append(round(mean(res_eval.mape_trening_res),3))
  dict_res['MAPE_trening_std'].append(round(stdev(res_eval.mape_trening_res),3))

# Lagring av resultater

Veiledet

In [ ]:
class Resultat_dicts():

  def __init__(self):

    self.dict_res_1_hel_opt = {"modell": [],
                               "test_andel": [],
                               "trening_andel": [],
                               "trening_markert_antall": [],
                               "tid_trening": [],
                               "parametere": [],
                               "RMSE_opt": [],
                               "RMSE_test": [],
                               "R2_test":[],
                               "MAE_test":[],
                               "MAPE_test":[],
                               "n_runder": [],
                               "n_splitt": [],
                               "tid_optimalisering": []}

    self.dict_res_2_hel_opt = copy.deepcopy(self.dict_res_1_hel_opt)
    self.dict_res_1_design_opt = copy.deepcopy(self.dict_res_1_hel_opt)
    self.dict_res_2_hel_opt = copy.deepcopy(self.dict_res_1_hel_opt)
    self.dict_res_2_design_opt = copy.deepcopy(self.dict_res_1_hel_opt)

    # Uten NIR
    self.dict_res_1_hel_opt_NIR = copy.deepcopy(self.dict_res_1_hel_opt)
    self.dict_res_1_design_opt_NIR = copy.deepcopy(self.dict_res_1_hel_opt)
    self.dict_res_2_hel_opt_NIR = copy.deepcopy(self.dict_res_1_hel_opt)
    self.dict_res_2_design_opt_NIR = copy.deepcopy(self.dict_res_1_hel_opt)

    self.dict_res_1_hel = {"modell": [],
                           "test_andel": [],
                           "trening_andel": [],
                           "trening_markert_antall": [],
                           "parametere": [],
                           "n_splitt": [],
                           "RMSE_opt": [],
                           "RMSE_test_snitt": [],
                           "RMSE_test_std": [],
                           "R2_test_snitt":[],
                           "R2_test_std":[],
                           "MAE_test_snitt":[],
                           "MAE_test_std":[],
                           "MAPE_test_snitt":[],
                           "MAPE_test_std":[],
                           "RMSE_val_snitt": [],
                           "RMSE_val_std": [],
                           "R2_val_snitt":[],
                           "R2_val_std":[],
                           "MAE_val_snitt":[],
                           "MAE_val_std":[],
                           "MAPE_val_snitt":[],
                           "MAPE_val_std":[],
                           "RMSE_trening_snitt": [],
                           "RMSE_trening_std": [],
                           "R2_trening_snitt":[],
                           "R2_trening_std":[],
                           "MAE_trening_snitt":[],
                           "MAE_trening_std":[],
                           "MAPE_trening_snitt":[],
                           "MAPE_trening_std":[]}

    self.dict_res_1_design = copy.deepcopy(self.dict_res_1_hel)
    self.dict_res_2_hel = copy.deepcopy(self.dict_res_1_hel)
    self.dict_res_2_design = copy.deepcopy(self.dict_res_1_hel)

    # Uten NIR
    self.dict_res_1_hel_NIR = copy.deepcopy(self.dict_res_1_hel)
    self.dict_res_1_design_NIR = copy.deepcopy(self.dict_res_1_hel)
    self.dict_res_2_hel_NIR = copy.deepcopy(self.dict_res_1_hel)
    self.dict_res_2_design_NIR = copy.deepcopy(self.dict_res_1_hel)

Semi-veiledet

In [ ]:
class Resultat_dicts_semi():

  def __init__(self):

    self.dict_res_1_hel_opt = {"modell": [],
                               "test_andel": [],
                               "trening_andel": [],
                               "trening_markert_antall": [],
                               "umarkert_antall": [],
                               "umarkert_benyttet": [],
                               "tid_trening": [],
                               "parametere": [],
                               "RMSE_opt": [],
                               "RMSE_test": [],
                               "R2_test":[],
                               "MAE_test":[],
                               "MAPE_test":[],
                               "n_runder": [],
                               "n_splitt": [],
                               "tid_optimalisering":[]}

    self.dict_res_2_hel_opt = copy.deepcopy(self.dict_res_1_hel_opt)
    self.dict_res_1_design_opt = copy.deepcopy(self.dict_res_1_hel_opt)
    self.dict_res_2_hel_opt = copy.deepcopy(self.dict_res_1_hel_opt)
    self.dict_res_2_design_opt = copy.deepcopy(self.dict_res_1_hel_opt)

    # Uten NIR
    self.dict_res_1_hel_opt_NIR = copy.deepcopy(self.dict_res_1_hel_opt)
    self.dict_res_1_design_opt_NIR = copy.deepcopy(self.dict_res_1_hel_opt)
    self.dict_res_2_hel_opt_NIR = copy.deepcopy(self.dict_res_1_hel_opt)
    self.dict_res_2_design_opt_NIR = copy.deepcopy(self.dict_res_1_hel_opt)

    self.dict_res_1_hel = {"modell": [],
                           "test_andel": [],
                           "trening_andel": [],
                           "trening_markert_antall": [],
                           "umarkert_antall": [],
                           "umarkert_benyttet": [],
                           "parametere": [],
                           "n_splitt": [],
                           "RMSE_opt": [],
                           "RMSE_test_snitt": [],
                           "RMSE_test_std": [],
                           "R2_test_snitt":[],
                           "R2_test_std":[],
                           "MAE_test_snitt":[],
                           "MAE_test_std":[],
                           "MAPE_test_snitt":[],
                           "MAPE_test_std":[],
                           "RMSE_val_snitt": [],
                           "RMSE_val_std": [],
                           "R2_val_snitt":[],
                           "R2_val_std":[],
                           "MAE_val_snitt":[],
                           "MAE_val_std":[],
                           "MAPE_val_snitt":[],
                           "MAPE_val_std":[],
                           "RMSE_trening_snitt": [],
                           "RMSE_trening_std": [],
                           "R2_trening_snitt":[],
                           "R2_trening_std":[],
                           "MAE_trening_snitt":[],
                           "MAE_trening_std":[],
                           "MAPE_trening_snitt":[],
                           "MAPE_trening_std":[]}

    self.dict_res_1_design = copy.deepcopy(self.dict_res_1_hel)
    self.dict_res_2_hel = copy.deepcopy(self.dict_res_1_hel)
    self.dict_res_2_design = copy.deepcopy(self.dict_res_1_hel)

    # Uten NIR
    self.dict_res_1_hel_NIR = copy.deepcopy(self.dict_res_1_hel)
    self.dict_res_1_design_NIR = copy.deepcopy(self.dict_res_1_hel)
    self.dict_res_2_hel_NIR = copy.deepcopy(self.dict_res_1_hel)
    self.dict_res_2_design_NIR = copy.deepcopy(self.dict_res_1_hel)